In [1]:
CACHED_TRAIN = True
CACHED_VAL = False

In [2]:
package_paths = [r"C:\Users\benja\Documents\projects\goalscorers"]
import sys
for path in package_paths:
    sys.path.append(path)
import goalscorer_package.constants as c
import goalscorer_package.data_cleaning as dc
import goalscorer_package.modelling as m
import datetime as dt
import pandas as pd
import numpy as np
import pymc as pm
print(f"Running on PyMC v{pm.__version__}")
import arviz as az
print(f"Running on Az v{az.__version__}")
import pytensor.tensor as pt
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
import glob
from statistics import mode
import pickle
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 1000)
pd.set_option("display.max_colwidth", 100)
pd.options.display.float_format = "{: ,.3f}".format
%matplotlib inline
plt.rcParams["figure.figsize"] = (10, 6)
%config InlineBackend.figure_formats = ["retina"]

Running on PyMC v5.6.1
Running on Az v0.16.0


# Hyper-Parameters

In [3]:
DICT_F_PRIOR = {
    "F": (0.0,  0.13),
}

In [4]:
NUM_CHAINS = 4

# Functions

## Data

In [5]:
def load_season(seasons: list[str], comp_ids: list[int]) -> pd.DataFrame:
    seasons_leagues = dc.get_seasons_leagues_from_str(seasons, comp_ids)
    df = dc.load_seasons_leagues_files("summary", True, seasons_leagues)
    return df    

In [6]:
def clean_df(df: pd.DataFrame) -> pd.DataFrame:
    df = dc.add_home(df)
    df = dc.add_opp_team(df)
    df = dc.add_frac_90(df)
    df = dc.add_league(df)
    df = dc.add_season(df)
    df["league_10"] = np.where((df.league == 10), 1, 0)
    df = dc.drop_na_npxg(df)
    df = dc.drop_na_frac_90(df)
    df = dc.add_npg(df)
    
    # Positions
    df = dc.split_positions(df)
    df = dc.position_to_generic_position(df)
    df = dc.drop_gk(df)

    # Time
    df = df.assign(
        epoch_datetime=lambda x: x.datetime,
        datetime=lambda x: pd.to_datetime(x.datetime, unit="s"),
        date=lambda x: x.datetime.dt.date,
    )
    df_matches = (
        df[["datetime", "home_team", "away_team"]]
        .drop_duplicates(ignore_index=True)
        .sort_values(["datetime", "home_team"], ignore_index=True)
        .assign(match_number=lambda x: range(1, len(x)+1))
    )
    df = (
        df
        .merge(df_matches, how="left", on=["datetime", "home_team", "away_team"], validate="m:1")
        .sort_values(["match_number", "home", "position"], ignore_index=True)
        .assign(time_interval=lambda x: pd.qcut(x.match_number, q=38))
    )
    df_interval_to_t = pd.DataFrame({"t": range(1, 39), "time_interval": np.sort(df.time_interval.unique())})
    df = df.merge(df_interval_to_t, how="left", on=["time_interval"], validate="m:1")


    return df 

In [7]:
def df_to_model_df(df: pd.DataFrame) -> pd.DataFrame:
    df_model = df[["npg", "npxg", "player_id", "position", "start", "league", "t"]].copy()
    return df_model

## Prior Specification

### Initial Prior

In [8]:
def first_player_prior_specification(df: pd.DataFrame, player_ids: list) -> pd.DataFrame:
    df_player_priors = (
        df
        .groupby("player_id", as_index=False)
        .agg(position=("position", mode))
        .assign(
            mu=DICT_F_PRIOR["F"][0],
            sigma=DICT_F_PRIOR["F"][1],
        )
        .set_index("player_id", drop=False)
        .loc[player_ids]
    )
    return df_player_priors

### Next Season Prior

In [9]:
def get_new_season_player_priors(df_post_player_priors: pd.DataFrame, df: pd.DataFrame, player_ids: list) -> pd.DataFrame:
    df_player_priors = (
        first_player_prior_specification(df, player_ids)
        .merge(df_post_player_priors, how="left", on=["player_id"], suffixes=["_default", "_post"], validate="1:1")
        .assign(
            mu_post=lambda x: x.mu_post.fillna(x.mu_default),
            sigma_post=lambda x: x.sigma_post.fillna(x.sigma_default),
        )
        [["player_id", "position_default", "mu_post", "sigma_post"]]
        .rename(columns={"mu_post": "mu", "sigma_post": "sigma", "position_default": "position"})
    )
    return df_player_priors

## Fit

In [10]:
def create_model(df_all: pd.DataFrame, df_model: pd.DataFrame, priors: tuple, player_id_codes: list, coords: dict) -> pm.model.Model:
    df_player_priors = priors
    
    with pm.Model(coords=coords) as model:
        # Data
        player_id_codes_data = pm.MutableData("player_id_codes_data", player_id_codes)
        npxg_data = pm.MutableData("npxg_data", df_model.npxg.values)
        npg_data = pm.MutableData("npg_data", df_model.npg.values)
    
        # Priors
        F = pm.Normal("F", mu=df_player_priors.mu.values, sigma=df_player_priors.sigma.values, dims="player_ids")
    
        # Deterministic transform
        exp = pm.Deterministic("exp", npxg_data * pt.exp(F[player_id_codes_data]))
    
        # Likelihood
        x = pm.Poisson(
            "x",
            mu=exp,
            observed=npg_data
        )
        
    return model

## Posterior

### Posterior to priors

In [11]:
def players_posterior_to_priors(df: pd.DataFrame, idata: az.InferenceData) -> pd.DataFrame:
    df_post_player_priors = (
        pd
        .DataFrame({"player_id": idata.posterior.F.player_ids})
        .merge(dc.calc_main_position(df), how="left", on=["player_id"], validate="1:1")
        .merge(dc.calc_main_squad(df), how="left", on=["player_id"], validate="1:1")
        .assign(
            mu=lambda x: [idata.posterior.F.sel(player_ids=player_id).data.mean() for player_id in x.player_id.values], 
            sigma=lambda x: [idata.posterior.F.sel(player_ids=player_id).data.std() for player_id in x.player_id.values], 
        )
    )
    return df_post_player_priors

### Utils

In [12]:
def increase_prior_variance(df_post_prior: pd.DataFrame, perc=10.0) -> pd.DataFrame:
    df_post_prior.sigma = df_post_prior.sigma * (1.0 + perc / 100.0)
    return df_post_prior

In [13]:
def get_df_posterior(df: pd.DataFrame, idata: az.InferenceData) -> pd.DataFrame:
    dict_bu = {"player_id": [], "F": []}

    for player_id in df.player_id.unique():
        F = idata.posterior.F.sel(player_ids=player_id).data.flatten().mean()

        dict_bu["player_id"].append(player_id)
        dict_bu["F"].append(F)

    df_posterior = (
        df
        .merge(pd.DataFrame(dict_bu), how="left", on=["player_id"], validate="m:1")
    )
    return df_posterior

## Prediction

In [14]:
def add_predictions_to_df(df_test: pd.DataFrame, idata: az.InferenceData) -> pd.DataFrame:
    df_players_f = pd.DataFrame({"player_id": df_test.player_id.unique()})
    for c in range(NUM_CHAINS):
        df_players_f[f"F_{c}"] = [np.array(idata.posterior.F.sel(player_ids=player_id, chain=c).data) for player_id in df_test.player_id.unique()]

    df_posterior_predictive = (
        df_test
        .copy()
        .merge(df_players_f, how="left", on=["player_id"], validate="m:1")
    )
    return df_posterior_predictive

In [15]:
def add_pred_to_df(df_test: pd.DataFrame) -> pd.DataFrame:
    df_test["pred_F"] = 0.0
    for c in range(NUM_CHAINS):
        df_test.pred_F += np.array([df_test[f"F_{c}"][n].mean() for n in range(len(df_test))]) / NUM_CHAINS

    return df_test

# Main

## TRAIN

In [16]:
post_priors_cache = dict()

In [17]:
train_seasons = ["2017-2018", "2018-2019", "2019-2020"]
train_comp_ids = [9, 10, 11, 12, 13, 20]

In [18]:
season = "2017-2018"

In [20]:
if not CACHED_TRAIN:
    for season in train_seasons:
        # Data
        df = load_season([season], train_comp_ids)
        df = clean_df(df)
        df_model = df_to_model_df(df)

        # Model data
        player_id_codes, player_ids = pd.factorize(df_model.loc[:, "player_id"], sort=True)

        # Prior specification
        if (season == train_seasons[0]):
            df_player_priors = first_player_prior_specification(df, player_ids)
        else:
            df_player_priors = get_new_season_player_priors(df_post_player_priors, df, player_ids)
        priors = df_player_priors

        # Model
        coords = {"player_ids": player_ids}
        model = create_model(df_model, df_model, priors, player_id_codes, coords)

        with model:
            idata = pm.sample()

        # Posterior priors
        df_posterior = get_df_posterior(df, idata)

        df_post_player_priors = players_posterior_to_priors(df, idata)

        df_post_player_priors = increase_prior_variance(df_post_player_priors)

        post_priors_cache[season] = {
            "df_posterior": df_posterior,
            "df_post_player_priors": df_post_player_priors,
        }
    
    with open(r"cache\post-priors-cache.pickle", "wb") as post_priors_cache_file:
        pickle.dump(post_priors_cache, post_priors_cache_file)

else:
    with open(r"cache\post-priors-cache.pickle", "rb") as post_priors_cache_file:
        post_priors_cache = pickle.load(post_priors_cache_file)

    post_priors_cache_final_season = post_priors_cache[train_seasons[-1]]

    df_posterior = post_priors_cache_final_season["df_posterior"]
    df_post_player_priors = post_priors_cache_final_season["df_post_player_priors"]

## VAL

In [21]:
def add_to_df_test(df_test: pd.DataFrame) -> pd.DataFrame:
    df_test = add_pred_to_df(df_test)
    return df_test

In [22]:
val_seasons = ["2020-2021"]
val_comp_ids = [9, 11, 12, 13, 20]

In [23]:
season = "2020-2021"

In [24]:
if not CACHED_VAL:
    df_test_preds = []
    # for season in val_seasons:
    df = load_season([season], val_comp_ids)
    df = clean_df(df)

    ts = np.sort(df.t.unique())
    ts = ts[1:]
    counter = 2
    for t in ts:
        # Train / Test
        df_train, df_test = df.query("t < @t").reset_index(drop=True), df.query("t == @t").reset_index(drop=True)
        df_model_train, df_model_test = df_to_model_df(df_train), df_to_model_df(df_test)
        df_model = pd.concat([df_model_train, df_model_test], ignore_index=True)

        # Model data
        player_id_codes, player_ids = pd.factorize(df_model.loc[:, "player_id"], sort=True)

        # Prior specification
        df_player_priors = get_new_season_player_priors(df_post_player_priors, df, player_ids)

        priors = df_player_priors

        # Train model
        coords = {"player_ids": player_ids}
        model = create_model(df_model, df_model, priors, player_id_codes, coords)

        with model:
            idata = pm.sample()

        # Test predictions
        df_test_pred = add_predictions_to_df(df_test, idata)
        df_test_preds.append(df_test_pred)

        print("Finished", counter, "/", len(ts) + 1)
        counter += 1
    
    # Test DataFrame
    df_test = pd.concat(df_test_preds, ignore_index=True)
    df_test = add_to_df_test(df_test)
    df_test.to_pickle(r"cache\df_test.pickle")

else:
    df_test = pd.read_pickle(r"cache\df_test.pickle")

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [F]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 34 seconds.


Finished 2 / 38


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [F]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 37 seconds.
The rhat statistic is larger than 1.01 for some parameters. This indicates problems during sampling. See https://arxiv.org/abs/1903.08008 for details


...

Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 189 seconds.
The rhat statistic is larger than 1.01 for some parameters. This indicates problems during sampling. See https://arxiv.org/abs/1903.08008 for details


Finished 37 / 38


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [F]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 199 seconds.
The rhat statistic is larger than 1.01 for some parameters. This indicates problems during sampling. See https://arxiv.org/abs/1903.08008 for details


Finished 38 / 38


In [25]:
az.summary(idata, "F").sort_values("mean", ascending=True)

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
F[bb4b1242],-0.175,0.147,-0.442,0.110,0.002,0.002,"8,402.000","2,597.000",1.010
F[16daec92],-0.168,0.160,-0.452,0.144,0.002,0.002,"6,429.000","2,608.000",1.000
F[3bb7f478],-0.158,0.111,-0.364,0.057,0.001,0.001,"8,434.000","2,644.000",1.000
F[93feac6e],-0.148,0.110,-0.364,0.055,0.001,0.001,"9,144.000","2,405.000",1.000
F[0ac94a23],-0.138,0.143,-0.400,0.140,0.002,0.002,"5,853.000","3,208.000",1.000
...,...,...,...,...,...,...,...,...,...
F[92e7e919],0.213,0.129,-0.025,0.457,0.001,0.001,"9,447.000","2,656.000",1.000
F[892d5bb1],0.215,0.131,-0.041,0.452,0.002,0.001,"7,218.000","2,376.000",1.000
F[eb2fe5b6],0.219,0.129,-0.033,0.462,0.001,0.001,"7,727.000","2,325.000",1.000
F[02c15616],0.242,0.160,-0.077,0.529,0.002,0.002,"9,508.000","2,320.000",1.000
